In [8]:
import gurobipy as gp
from gurobipy import GRB
import SystemCharacteristics as data
import PlotsRestaurant
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
fixed_data = data.get_fixed_data()
fixed_data.keys()

dict_keys(['num_timeslots', 'initial_temperature', 'previous_initial_temperature', 'initial_humidity', 'heating_max_power', 'heat_exchange_coeff', 'heating_efficiency_coeff', 'thermal_loss_coeff', 'heat_vent_coeff', 'heat_occupancy_coeff', 'temp_min_comfort_threshold', 'temp_OK_threshold', 'temp_max_comfort_threshold', 'outdoor_temperature', 'vent_min_up_time', 'humidity_threshold', 'ventilation_power', 'humidity_occupancy_coeff', 'humidity_vent_coeff'])

In [15]:
price = pd.read_csv('PriceData.csv').iloc[0, :].tolist()
price

[3.48812593193225,
 2.237250383112821,
 1.1091079406719566,
 0.5295048776649726,
 0.5480024850667341,
 0.2832462843032866,
 1.016859658699676,
 1.7412889228068242,
 2.587035092277435,
 2.9889689598133167]

In [ ]:
row_nr = 0

# Load Data from CSVs as requested
price = pd.read_csv('PriceData.csv').iloc[row_nr, :].tolist()
Occ_r1 = pd.read_csv('OccupancyRoom1.csv').iloc[row_nr, :].tolist()
Occ_r2 = pd.read_csv('OccupancyRoom2.csv').iloc[row_nr, :].tolist()

# Define T based on fixed_data
T = fixed_data['num_timeslots']
R = [0, 1]  # Room indices (Room 1 and Room 2)

# Initialize Model
model = gp.Model("Restaurant_Energy_Optimization")

# --- Decision Variables ---
# Temperature in room 1 and 2 [cite: 16, 49, 59]
Temp_r1 = model.addVars(T + 1, lb=-GRB.INFINITY, name="Temp_r1")
Temp_r2 = model.addVars(T + 1, lb=-GRB.INFINITY, name="Temp_r2")

# Heater power consumption for room 1 and 2 [cite: 14, 53, 79]
h_r1 = model.addVars(T, lb=0, ub=fixed_data['heating_max_power'], name="h_r1")
h_r2 = model.addVars(T, lb=0, ub=fixed_data['heating_max_power'], name="h_r2")

# Ventilation status (Binary) [cite: 13, 60, 80]
v = model.addVars(T, vtype=GRB.BINARY, name="v")

# Humidity [cite: 17, 47, 58]
Hum = model.addVars(T + 1, lb=0, ub=100, name="Hum")

# Auxiliary variables for constraints (Inertia, Overrules) [cite: 18, 51, 61]
s = model.addVars(T, vtype=GRB.BINARY, name="s") # Start-up variable for ventilation
z_low = model.addVars(R, T, vtype=GRB.BINARY, name="z_low") # Low temp overrule
z_high = model.addVars(R, T, vtype=GRB.BINARY, name="z_high") # High temp overrule

# [cite_start]--- 5. Initial Conditions [cite: 46, 48, 54] ---
model.addConstr(Temp_r1[0] == fixed_data['initial_temperature'], name="init_temp_r1")
model.addConstr(Temp_r2[0] == fixed_data['initial_temperature'], name="init_temp_r2")
model.addConstr(Hum[0] == fixed_data['initial_humidity'], name="init_hum")
model.addConstr(v[0] == 0, name="init_vent")

# [cite_start]--- 6. Objective Function [cite: 43] ---
model.setObjective(
    gp.quicksum(
        (fixed_data['ventilation_power'] * v[t] + h_r1[t] + h_r2[t]) * price[t-1] 
        for t in T_range
    ), GRB.MINIMIZE
)

# --- 7. Constraints ---

# [cite_start]Humidity Dynamics [cite: 47]
model.addConstrs((Hum[t] == Hum[t-1] + fixed_data['humidity_occupancy_coeff'] * (Occ_r1[t-1] + Occ_r2[t-1]) - 
                  fixed_data['humidity_vent_coeff'] * v[t-1] for t in T_range), name="Hum_Dynamics")

# [cite_start]Temperature Dynamics Room 1 [cite: 49]
model.addConstrs((Temp_r1[t] == Temp_r1[t-1] + fixed_data['heat_exchange_coeff'] * (Temp_r2[t-1] - Temp_r1[t-1]) +
                  fixed_data['thermal_loss_coeff'] * (fixed_data['outdoor_temperature'][t-1] - Temp_r1[t-1]) +
                  fixed_data['heating_efficiency_coeff'] * h_r1[t] - fixed_data['heat_vent_coeff'] * v[t] + 
                  fixed_data['heat_occupancy_coeff'] * Occ_r1[t-1] for t in T_range), name="Temp_Dynamics_R1")

# [cite_start]Temperature Dynamics Room 2 [cite: 49]
model.addConstrs((Temp_r2[t] == Temp_r2[t-1] + fixed_data['heat_exchange_coeff'] * (Temp_r1[t-1] - Temp_r2[t-1]) +
                  fixed_data['thermal_loss_coeff'] * (fixed_data['outdoor_temperature'][t-1] - Temp_r2[t-1]) +
                  fixed_data['heating_efficiency_coeff'] * h_r2[t] - fixed_data['heat_vent_coeff'] * v[t] + 
                  fixed_data['heat_occupancy_coeff'] * Occ_r2[t-1] for t in T_range), name="Temp_Dynamics_R2")

# [cite_start]Low-temperature overrule [cite: 51, 53]
model.addConstrs((fixed_data['temp_min_comfort_threshold'] - Temp_r1[t] <= M * z_low[1, t] for t in T_range), name="Low_Temp_R1_Bound")
model.addConstrs((fixed_data['temp_min_comfort_threshold'] - Temp_r2[t] <= M * z_low[2, t] for t in T_range), name="Low_Temp_R2_Bound")
model.addConstrs((h_r1[t] >= z_low[1, t] * fixed_data['heating_max_power'] for t in T_range), name="Low_Temp_R1_Power")
model.addConstrs((h_r2[t] >= z_low[2, t] * fixed_data['heating_max_power'] for t in T_range), name="Low_Temp_R2_Power")

# [cite_start]Low-temperature inertia (Only for t > 1) [cite: 51]
model.addConstrs((z_low[1, t] >= z_low[1, t-1] - 1 + (fixed_data['temp_OK_threshold'] - Temp_r1[t]) / M 
                  for t in range(2, T + 1)), name="Low_Temp_R1_Inertia")
model.addConstrs((z_low[2, t] >= z_low[2, t-1] - 1 + (fixed_data['temp_OK_threshold'] - Temp_r2[t]) / M 
                  for t in range(2, T + 1)), name="Low_Temp_R2_Inertia")

# [cite_start]High-temperature overrule [cite: 53]
model.addConstrs((Temp_r1[t] - fixed_data['temp_max_comfort_threshold'] <= M * z_high[1, t] for t in T_range), name="High_Temp_R1_Bound")
model.addConstrs((Temp_r2[t] - fixed_data['temp_max_comfort_threshold'] <= M * z_high[2, t] for t in T_range), name="High_Temp_R2_Bound")
model.addConstrs((h_r1[t] <= (1 - z_high[1, t]) * fixed_data['heating_max_power'] for t in T_range), name="High_Temp_R1_Power")
model.addConstrs((h_r2[t] <= (1 - z_high[2, t]) * fixed_data['heating_max_power'] for t in T_range), name="High_Temp_R2_Power")

# [cite_start]Humidity Overrule Controller [cite: 53]
model.addConstrs((Hum[t] <= fixed_data['humidity_threshold'] + M * v[t] for t in T_range), name="Hum_Overrule")

# [cite_start]Ventilation System Inertia [cite: 55, 56]
model.addConstrs((s[t] >= v[t] - v[t-1] for t in T_range), name="Vent_Startup")
model.addConstrs((v[t] + v[t+1] + v[t+2] >= 3 * s[t] for t in range(1, T - 1)), name="Vent_Min_Up_Time")

# Solve
model.optimize()

TypeError: unsupported operand type(s) for -: 'list' and 'float'